In [1]:
import pandas as pd
import numpy as np
import os, gc
os.chdir("/home/jovyan/work/phd/datasets/gtex/10/")

In [3]:
df=pd.read_csv("mainTable_hv.csv", index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2999 entries, ENSG00000268903.1 to ENSG00000210196.2
Columns: 1000 entries, GTEX-1GZ2Q-0011-R6a-SM-9OSWM to GTEX-RM2N-0526-SM-2TF4N
dtypes: float64(1000)
memory usage: 22.9+ MB


In [4]:
with open("corpus.txt",'w') as file:
    for sample in df.columns:
        for g in np.array(df[sample].sort_values(ascending=False).index[:1000],dtype=str):
            file.write(g[:15])
            file.write(" ")
        file.write("\n")

In [ ]:
gc.collect()

In [ ]:
os.system("topicmap -f corpus.txt -r 10 -t 10 -seed 42 -o tm")

In [5]:
df_words = pd.read_csv("tm/word_wn_count.txt", sep=' ', header=None)
df_words.columns=['word', 'word-id', 'occurrence']
df_words.sort_values('word-id')

,word,word-id,occurrence
2775,ENSG00000087250,0,959
2848,ENSG00000127585,1,982
2813,ENSG00000178531,2,970
2779,ENSG00000131771,3,961
2902,ENSG00000053438,4,993
...,...,...,...
33,ENSG00000284049,2994,3
80,ENSG00000237020,2995,8
9,ENSG00000207773,2996,1
10,ENSG00000282912,2997,1


In [6]:
df_topic_distr = pd.read_csv("tm/lda_gammas_final.txt", sep=' ', header=None)
df_topic_distr.columns=['Topic %d'%(t+1) for t in df_topic_distr.columns]
df_topic_distr.index.name='i_doc'
df_topic_distr.insert(0,'doc',df.columns)
df_topic_distr=df_topic_distr.dropna(how='all',axis=1)
df_topic_distr

,doc,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8
i_doc,,,,,,,,,
0,GTEX-1GZ2Q-0011-R6a-SM-9OSWM,1000.020000,0.025311,0.019659,0.030359,0.0152,0.018537,0.041688,0.036664
1,GTEX-RU72-0011-R5A-SM-2TF6U,1000.020000,0.025311,0.019659,0.030359,0.0152,0.018537,0.041688,0.036664
2,GTEX-WHSE-0011-R3A-SM-3P5ZM,1000.020000,0.025311,0.019659,0.030359,0.0152,0.018537,0.041688,0.036664
3,GTEX-14PQA-0011-R1b-SM-6EU1G,1000.020000,0.025311,0.019659,0.030359,0.0152,0.018537,0.041688,0.036664
4,GTEX-13CF2-0011-R1a-SM-5LZY1,851.932000,0.025311,0.019659,0.030359,62.0553,86.065500,0.041688,0.036664
...,...,...,...,...,...,...,...,...,...
995,GTEX-17HG3-0226-SM-7EWEA,0.019399,362.762000,0.019659,0.030359,0.0152,637.282000,0.041688,0.036664
996,GTEX-139T6-0326-SM-5J2LY,0.019399,0.025311,0.019659,0.030359,0.0152,1000.020000,0.041688,0.036664
997,GTEX-1PDJ9-0826-SM-E76PS,0.019399,62.456600,0.019659,0.030359,0.0152,937.587000,0.041688,0.036664


In [7]:
clusters = df_topic_distr.drop('doc',1).values.argmax(1)
df_clusters=pd.DataFrame()
for cluster in range(np.max(clusters)+1):
    elems=df.columns[clusters==cluster].values
    df_clusters.insert(0,'Cluster %d'%(cluster+1),np.concatenate([elems, ['' for _ in range(len(df.columns)- len(elems))]]))

In [8]:
df_clusters.sort_index(axis=1).to_csv("tm/tm_level_0_clusters.csv", index=False, header=True)

In [9]:
df_topic_distr.to_csv("tm/tm_level_0_topic-dist.csv", index=True, header=True)

In [10]:
df_word_distr = pd.DataFrame().fillna(0)
with open("tm/lda_betas_sparse_final.txt","r") as f:
    for line in f.read().split("\n"):
        row = line.split(" ")
        if len(row) < 2:
            continue
        topic = int(row[0])+1
        line=np.array(row[1:-1], dtype=float).reshape(int((len(row)-1)/2),2)
        for el in line:
            df_word_distr.at[df_words[df_words['word-id']==int(el[0])].word.values[0], f"Topic {topic}"] = el[1]
#df_word_distr.index=df_words['word']
df_word_distr.fillna(0)
df_word_distr.to_csv("tm/tm_level_0_word-dist.csv", index=True, header=True)